In [29]:
from src.unigram import UnigramTokenizer
tokenizer = UnigramTokenizer(corpus_path='text_mag.txt')
vocab, wordcount = tokenizer.init_word_count()


Start create seed Vocab. 
start word count
start substring count
17.140061532987676
Seed Vocab Completed. 


In [30]:
print(vocab['a'])
print(vocab['b'])
print(vocab['xx'])
print(vocab['Martello'])
print(vocab['years'])
print(vocab['cannot'])
print(vocab['s'])

4.066539721073805
5.811912635886514
16.041449244319566
16.44691435242773
9.763553406661456
16.44691435242773
4.266970099727252


In [31]:
from collections import defaultdict
import igraph as ig

class TreeNode:
    def __init__(self, val, depth, left, right, score):
        self.val = val
        self.left = left
        self.right = right
        self.depth = depth
        self.score = score

def span_scores(s):
    if s == ' ':
        return 50
    if s[0] == ' ':
        s = s[1:]
    if s not in vocab:
        return 17.140061532987676
    else:
        return vocab[s]

def cky_parse(sentence, span_scores):
    n = len(sentence)
    dp = defaultdict(lambda: defaultdict(int))
    bp = defaultdict(TreeNode)

    # Initialize the diagonal entries
    for i in range(n):
        char = sentence[i]
        dp[i][i + 1] = span_scores(sentence[i:i + 1])
        bp[(i,i+1)] = TreeNode(sentence[i:i + 1], 1, None, None, dp[i][i + 1])

    # Apply the CKY algorithm
    for span in range(2, n + 1):
        for start in range(n - span + 1):
            end = start + span
            for split in range(start + 1, end):
                score = dp[start][split] + dp[split][end] + span_scores(sentence[start:end])
                if score < dp[start][end] or dp[start][end] == 0.0:
                    dp[start][end] = score
                    bp[(start,end)] = TreeNode(sentence[start:end], span, bp[(start,split)], bp[(split,end)], score)

    # Find the best parse tree
    best_parse = bp[(0,n)]
    return best_parse

def plot_parse_tree(parse_tree):
    # Create the igraph object
    g = ig.Graph(directed=True)
    nodes = []
    edges = []

    # Add nodes to the graph
    def add_node(node):
        if node is None:
            return
        node_id = len(nodes)
        nodes.append(node)
        g.add_vertex(node.val)
        if node.left:
            add_node(node.left)
            edges.append((node_id, len(nodes) - 1))
        if node.right:
            add_node(node.right)
            edges.append((node_id, len(nodes) - 1))

    add_node(parse_tree)

    # Add edges to the graph
    g.add_edges(edges)

    # Plot the graph
    layout = g.layout("tree")
    plot = ig.plot(g, layout=layout, vertex_label=g.vs["name"], bbox=(1000, 1000), margin=20)
    plot.save("parsetree.png")

# Example usage
sentence = "The mid-2000s unstoppable cannot makes differences."

parse_tree = cky_parse(sentence, span_scores)
plot_parse_tree(parse_tree)

In [54]:
from transformers import GPT2Tokenizer, GPT2Model
from datasets import load_dataset, load_metric
import torch
import numpy as np

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
# model = GPT2Model.from_pretrained('gpt2')

In [81]:
text = "The"
print(tokenizer.convert_tokens_to_ids("cann"))
print(tokenizer.convert_ids_to_tokens(28))
# Encode the text, which converts the text to a sequence of token IDs
encoded_text = tokenizer.encode(text=text, add_special_tokens=True)
print(encoded_text)
decoded_text = tokenizer.decode(encoded_text, skip_special_tokens=True)
print(decoded_text)

50256
=
[464]
The


((During)(( the)( bubble years((( o)f)(((( the)( mid-))2000)(s.)))))))